In [1]:
import argparse
import os
import random

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import gradio as gr

from video_llama.common.config import Config
from video_llama.common.dist_utils import get_rank
from video_llama.common.registry import registry
from video_llama.conversation.conversation_video import (
    Chat, Conversation, default_conversation,SeparatorStyle,conv_llava_llama_2
)
import decord
decord.bridge.set_bridge('torch')

/users/piyush/install/miniconda3/envs/videollama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/users/piyush/install/miniconda3/envs/videollama/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/users/piyush/install/miniconda3/envs/videollama/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [2]:
from video_llama.datasets.builders import *
from video_llama.models import *
from video_llama.processors import *
from video_llama.runners import *
from video_llama.tasks import *

In [3]:
def setup_seeds(config):
    seed = config.run_cfg.seed + get_rank()

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    cudnn.benchmark = False
    cudnn.deterministic = True

In [4]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [5]:
args = dict(
    cfg_path="./eval_configs/video_llama_eval_only_vl_edited.yaml",
    model_type="llama_v2",
    gpu_id=0,
    options=[],
)
args = AttrDict(args)
args

{'cfg_path': './eval_configs/video_llama_eval_only_vl_edited.yaml',
 'model_type': 'llama_v2',
 'gpu_id': 0,
 'options': []}

In [6]:
cfg = Config(args)

In [7]:
model_config = cfg.model_cfg
model_config.device_8bit = args.gpu_id
model_cls = registry.get_model_class(model_config.arch)


In [8]:
model_config.llama_model

'/work/piyush/pretrained_checkpoints/LargeModels/VideoLLAMA/Video-LLaMA-2-7B-Pretrained/llama-2-7b-chat-hf/'

In [9]:
model_config.ckpt

'/work/piyush/pretrained_checkpoints/LargeModels/VideoLLAMA/Video-LLaMA-2-7B-Pretrained/VL_LLaMA_2_7B_Pretrained.pth'

In [10]:
# local_folder = "/users/piyush/.cache/huggingface/hub/models--DAMO-NLP-SG--Video-LLaMA-2-7B-Pretrained/"\
#     "snapshots/52407e33d301c6fbab629c4a98391905a02e849b/llama-2-7b-chat-hf"
# model_config.llama_model = "meta-llama/Llama-2-7b-chat-hf"
# local_folder = "/work/piyush/pretrained_checkpoints/LargeModels/VideoLLAMA/Video-LLaMA-2-7B-Pretrained/llama-2-7b-chat-hf/"
# model_config.llama_model = local_folder

In [11]:
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))
model = model.eval()

/users/piyush/install/miniconda3/envs/videollama/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading VIT
Loading VIT Done
Loading Q-Former


Using pad_token, but it is not set yet.
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.78s/it]


Load first Checkpoint: /work/piyush/pretrained_checkpoints/LargeModels/VideoLLAMA/Video-LLaMA-2-7B-Pretrained/VL_LLaMA_2_7B_Pretrained.pth


In [12]:
vis_processor_cfg = cfg.datasets_cfg.webvid.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)

In [14]:
chat = Chat(model, vis_processor, device='cuda:{}'.format(args.gpu_id))

In [16]:
dir(chat)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'answer',
 'ask',
 'device',
 'get_context_emb',
 'image_vis_processor',
 'model',
 'upload_img',
 'upload_video',
 'upload_video_without_audio',
 'vis_processor']

In [19]:
n_params = np.sum([p.numel() for p in model.parameters()])
n_params / 1e6

7851.579264